### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [6]:
import pandas as pd
import json
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [58]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv")
coram_df = pd.read_csv("data/prediction_data/coram.csv")
fact_themes_df = pd.read_csv("data/prediction_data/fact_themes.csv")
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv")
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv")
# Load the JSON file into a dictionary
# with open('data/prediction_data/issues.json') as f:
#     issues_data = [json.loads(line) for line in f]
# issues_df = pd.DataFrame(issues_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, coram_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, fact_themes_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, sg_legal_cases_df, left_on='casename', right_on='filename', how='outer')
# merged_df = pd.merge(merged_df, issues_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')

merged_df.drop(columns=['Unnamed: 0'], inplace=True)
merged_df.drop(columns=['filename'], inplace=True)

# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

                                           Coram  themes court_level  \
0  ['Chao Hick Tin , L P Thean , Yong Pung How']     5.0        SGCA   
1   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
2   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
3   ['Chao Hick Tin , Tan Lee Meng , L P Thean']     5.0        SGCA   
4   ['Chao Hick Tin , Lai Kew Chai , L P Thean']    12.0        SGCA   

         target  
0    Favourable  
1    Favourable  
2    No outcome  
3  Unfavourable  
4  Unfavourable  


### Data Preprocessing

In [59]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

casename        0
area_of_law     0
Coram           7
themes         47
court_level     0
target         47
dtype: int64
              casename area_of_law  \
241  2000_SGHC_257.pdf          []   
274  2000_SGHC_290.pdf          []   
412   2001_SGCA_66.pdf          []   
432  2001_SGHC_101.pdf          []   
438  2001_SGHC_108.pdf          []   
442  2001_SGHC_111.pdf          []   
448  2001_SGHC_118.pdf          []   
457  2001_SGHC_128.pdf          []   
460  2001_SGHC_130.pdf          []   
462  2001_SGHC_132.pdf          []   
475  2001_SGHC_148.pdf          []   
478  2001_SGHC_150.pdf          []   
479  2001_SGHC_151.pdf          []   
489  2001_SGHC_163.pdf          []   
498  2001_SGHC_174.pdf          []   
536  2001_SGHC_214.pdf          []   
537  2001_SGHC_215.pdf          []   
544  2001_SGHC_222.pdf          []   
546  2001_SGHC_224.pdf          []   
550  2001_SGHC_228.pdf          []   
551  2001_SGHC_229.pdf          []   
555  2001_SGHC_232.pdf          []   
564  

In [60]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,Coram,themes,court_level,target
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...","[Chao Hick Tin , L P Thean , Yong Pung How]",5.0,SGCA,Favourable
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,Favourable
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...","[Chao Hick Tin , Tan Lee Meng , L P Thean]",5.0,SGCA,No outcome


### Flatten areas_of_law

In [61]:
all_areas = []

for index, row in merged_df.iterrows():

    areas = row['area_of_law']
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        flat_areas.extend(sub_areas)
    all_areas.append(flat_areas)

### One hot encoding

In [69]:
# one-hot encode aol
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(all_areas)

binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)

processed_df = pd.concat([merged_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

           casename                                        Coram  themes  \
0   2000_SGCA_1.pdf  [Chao Hick Tin , L P Thean , Yong Pung How]     5.0   
1  2000_SGCA_10.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   
2  2000_SGCA_11.pdf   [Chao Hick Tin , Tan Lee Meng , L P Thean]     5.0   

  court_level      target  "a larger sum being repaid"  "abet"  \
0        SGCA  Favourable                          0.0     0.0   
1        SGCA  Favourable                          0.0     0.0   
2        SGCA  No outcome                          0.0     0.0   

   "an interest in any matter"  "any action proposed or contemplated"  \
0                          0.0                                    0.0   
1                          0.0                                    0.0   
2                          0.0                                    0.0   

   "any person"  ...  writ of  seizure and sale  writ of summons  \
0           0.0  ...                        0.0              0.0   
1

In [70]:
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]

In [71]:
# one-hot encode coram
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])

binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)

processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename  themes court_level        target  \
0   2000_SGCA_1.pdf     5.0        SGCA    Favourable   
1  2000_SGCA_10.pdf     5.0        SGCA    Favourable   
2  2000_SGCA_11.pdf     5.0        SGCA    No outcome   
3  2000_SGCA_12.pdf     5.0        SGCA  Unfavourable   
4  2000_SGCA_13.pdf    12.0        SGCA  Unfavourable   

   "a larger sum being repaid"  "abet"  "an interest in any matter"  \
0                          0.0     0.0                          0.0   
1                          0.0     0.0                          0.0   
2                          0.0     0.0                          0.0   
3                          0.0     0.0                          0.0   
4                          0.0     0.0                          0.0   

   "any action proposed or contemplated"  "any person"  "appeal"  ...  \
0                                    0.0           0.0       0.0  ...   
1                                    0.0           0.0       0.0  ...   
2          

#### splitting

In [42]:
X = merged_df.drop(columns=['target'])
y = merged_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

### Feature Engineering

In [ ]:
## One hot encoding
## vector embedding
## pipeline

### Modeling

In [ ]:
# Perform modelling

### Evaluation

In [ ]:
## Perform Evaluation